In [ ]:
#import libraries
!pip install tmdbv3api
import pandas as pd
import numpy as np
from tmdbv3api import TMDb
import json
import requests
from tmdbv3api import Movie

#My TMDB API KEY
tmdb = TMDb()
tmdb.api_key = '946350b18b862d23ce1763b947a7e3e1'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Functions for data collection and preprocessing

In [ ]:
#function for fetching movie id
from tmdbv3api import Movie
tmdb_movie = Movie() 

def get_id(x):
    result = tmdb_movie.search(x)
    if not result:
      return np.NaN
    else:
      movie_id = 0
      for i in range(len(result)):
        if(result[i]['original_language'] == "hi"):
          movie_id = result[i]['id']
          break
      return movie_id

In [ ]:
#function for fetching genres
def get_genre(x):
    genres = []
    result = tmdb_movie.search(x)
    if not result:
      return np.NaN
      
    else:
      try:
        movie_id = get_id(x)  
        response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
        data_json = response.json()
        if data_json['genres']:
            genre_str = " " 
            for i in range(0,len(data_json['genres'])):
                genres.append(data_json['genres'][i]['name'])
            return genre_str.join(genres)
        else:
            return np.NaN
      except:
        return np.NaN

In [ ]:
#function for fetching keywords
def get_keywords(x):
    keywords = []
    result = tmdb_movie.search(x)
    
    if not result:
      return np.NaN
    else:
      try:
        movie_id = get_id(x)   
        response = requests.get('https://api.themoviedb.org/3/movie/{}/keywords?api_key={}'.format(movie_id,tmdb.api_key))
        data_json = response.json()
        if data_json['keywords']:
            keywords_str = " " 
            for i in range(0,len(data_json['keywords'])):
                keywords.append(data_json['keywords'][i]['name'])
            return keywords_str.join(keywords)
        else:
            return np.NaN
      except:
        return np.NaN

In [ ]:
get_keywords('boss')

'sibling relationship remake'

In [ ]:
#function for fetching overview score of movie
def get_overview(x):
    result = tmdb_movie.search(x)
    if not result:
      return np.NaN
    else:
      try:
        movie_id = get_id(x)     
        response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
        data_json = response.json()
        if data_json['overview']:
            return data_json['overview']
        else:
            return np.NaN
      except:
        return np.NaN


In [ ]:
#function for fetching overview score of movie
def get_tagline(x):
    result = tmdb_movie.search(x)
    if not result:
      return np.NaN
    else:
      try:
        movie_id = get_id(x)     
        response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
        data_json = response.json()
        if data_json['tagline']:
            return data_json['tagline']
        else:
            return np.NaN
      except:
        return np.NaN

In [ ]:
#function for fetching popularity score of movie
def get_popularity(x):
    result = tmdb_movie.search(x)
    if not result:
      return np.NaN
    else:
      try:
        movie_id = get_id(x)    
        response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
        data_json = response.json()
        if data_json['popularity']:
            return data_json['popularity']
        else:
            return np.NaN
      except:
        return np.NaN

In [ ]:
#function for fetching vote average of movie
def get_vote_average(x):
    result = tmdb_movie.search(x)
    if not result:
      return np.NaN
    else:
      try:
        movie_id = get_id(x)     
        response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
        data_json = response.json()
        if data_json['vote_average']:
            return data_json['vote_average']
        else:
            return np.NaN
      except:
        return np.NaN

In [ ]:
#function for fetching popularity score of movie
def get_vote_count(x):
    result = tmdb_movie.search(x)
    if not result:
      return np.NaN
    else:
      try:
        movie_id = get_id(x)     
        response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
        data_json = response.json()
        if data_json['vote_count']:
            return data_json['vote_count']
        else:
            return np.NaN
      except:
        return np.NaN

In [ ]:
#functions for fetching cast name
def get_actor1(x):
    if type(x)==float:
        return np.NaN
    return ((x.split(", ")[0]))

def get_actor2(x):
    
    if type(x)==float or len(x.split(", ")) < 2:
        return np.NaN
    else:
        return (x.split(", ")[1])

def get_actor3(x):
    if type(x)==float or len(x.split(", ")) < 3:
        return np.NaN
    else:
        return (x.split(", ")[2])

In [ ]:
def preprocess_data(df):
    
    #if row has NULL entries we remove it from data
    df.dropna(inplace=True, thresh=13)
    
    df = df.rename(columns={'Title':'movie_title','Director': 'director', 'Genre':'genres'})
    df = df.loc[:,['id','movie_title','keywords','tagline','overview','director','actor_1','actor_2','actor_3','genres','popularity', 'year', 'vote_average','vote_count']]
    
    df['genres'] = df['genres'].replace(np.nan, 'na')
    df['overview'] = df['overview'].replace(np.nan, 'na')
    df['tagline'] = df['tagline'].replace(np.nan, 'na')
    df['keywords'] = df['keywords'].replace(np.nan, 'na')
    df['director'] = df['director'].replace(np.nan, 'unknown')
    df['actor_1'] = df['actor_1'].replace(np.nan, 'unknown')
    df['actor_2'] = df['actor_2'].replace(np.nan, 'unknown')
    df['actor_3'] = df['actor_3'].replace(np.nan, 'unknown')
    df['movie_title'] = df['movie_title'].str.lower()
    df['describe'] = df['keywords'] + ' ' + df['tagline'] + ' ' +df['overview'] + ' '+df['actor_1'] + ' ' + df['actor_2'] + ' '+ df['actor_3'] + ' '+ df['director'] +' ' + df['genres'] + ' ' + df['vote_count'].map(str)
    
    return df

In [ ]:
#function for collecting data of bollywood movies
def get_movie_data(year):
    #get movie data from wikipedia
    link = "https://en.wikipedia.org/wiki/List_of_Hindi_films_of_"+year
    df1 = pd.read_html(link, header=0)[2]
    df2 = pd.read_html(link, header=0)[3]
    df3 = pd.read_html(link, header=0)[4]
    df4 = pd.read_html(link, header=0)[5]
    df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)
    
    #fetch genres, popularity of the movie through tmdb api key
    tmdb = TMDb()
    tmdb.api_key = '946350b18b862d23ce1763b947a7e3e1'
    tmdb_movie = Movie() 
    
    #fetch movie id
    df['id'] = df['Title'].map(lambda x: get_id(str(x)))
    
    #fetch keywords
    df['keywords'] = df['Title'].map(lambda x: get_keywords(str(x)))
    
    #fetch tagline
    df['tagline'] = df['Title'].map(lambda x: get_tagline(str(x)))
    
    #fetch overview
    df['overview'] = df['Title'].map(lambda x: get_overview(str(x)))
    
    #fetch Genre
    df['Genre'] = df['Title'].map(lambda x: get_genre(str(x)))
    
    #fetch poplurity
    df['popularity'] = df['Title'].map(lambda x: get_popularity(str(x)))
    
    #fetch vote average
    df['vote_average'] = df['Title'].map(lambda x: get_vote_average(str(x)))
    
    #fetch vote count
    df['vote_count'] = df['Title'].map(lambda x: get_vote_count(str(x)))
    
    #collect required features
    df = df[['id','Title','keywords', 'tagline', 'overview','Director','Cast','Genre','popularity', 'vote_average', 'vote_count']]
    
    #add actors name and release year of the movie
    df['actor_1'] = df['Cast'].map(lambda x: get_actor1(x))
    df['actor_2'] = df['Cast'].map(lambda x: get_actor2(x))
    df['actor_3'] = df['Cast'].map(lambda x: get_actor3(x))
    df['year'] = year

    df = df.rename(columns={'Title':'movie_title','Director': 'director', 'Genre':'genres'})
    df = df.loc[:,['id','movie_title','keywords','tagline','overview','director','actor_1','actor_2','actor_3','genres','popularity', 'year', 'vote_average','vote_count']]
    
    #data preprocess
    #data = preprocess_data(df)
    
    return df

## Collecting Movies data released after 2000

In [ ]:
#collecting movies of 2000
data = get_movie_data('2000')

In [ ]:
#collecting movies of 2001
data_2001 = get_movie_data('2001')
data = data.append(data_2001,ignore_index=True)

In [ ]:
#collecting movies of 2002
data_2002 = get_movie_data('2002')
data = data.append(data_2002,ignore_index=True)

In [ ]:
#collecting movies of 2003
data_2003 = get_movie_data('2003')
data = data.append(data_2003,ignore_index=True)

In [ ]:
#collecting movies of 2004
data_2004 = get_movie_data('2004')
data = data.append(data_2004,ignore_index=True)

In [ ]:
#collecting movies of 2005
data_2005 = get_movie_data('2005')
data = data.append(data_2005,ignore_index=True)

In [ ]:
#collecting movies of 2006
data_2006 = get_movie_data('2006')
data = data.append(data_2006,ignore_index=True)

In [ ]:
#collecting movies of 2007
data_2007 = get_movie_data('2007')
data = data.append(data_2007,ignore_index=True)

In [ ]:
#collecting movies of 2008
data_2008 = get_movie_data('2008')
data = data.append(data_2008,ignore_index=True)

In [ ]:
#collecting movies of 2009
data_2009 = get_movie_data('2009')
data = data.append(data_2009,ignore_index=True)

In [ ]:
#collecting movies of 2010
data_2010 = get_movie_data('2010')
data = data.append(data_2010,ignore_index=True)

In [ ]:
#collecting movies of 2011
data_2011 = get_movie_data('2011')
data = data.append(data_2011,ignore_index=True)

In [ ]:
#collecting movies of 2012
data_2012 = get_movie_data('2012')
data = data.append(data_2012,ignore_index=True)

In [ ]:
#collecting movies of 2013
data_2013 = get_movie_data('2013')
data = data.append(data_2013,ignore_index=True)

In [ ]:
#collecting movies of 2014
data_2014 = get_movie_data('2014')
data = data.append(data_2014,ignore_index=True)

In [ ]:
#collecting movies of 2015
data_2015 = get_movie_data('2015')
data = data.append(data_2015,ignore_index=True)

In [ ]:
#collecting movies of 2016
data_2016 = get_movie_data('2016')
data = data.append(data_2016,ignore_index=True)

In [ ]:
#collecting movies of 2017
data_2017 = get_movie_data('2017')
data = data.append(data_2017,ignore_index=True)

In [ ]:
#collecting movies of 2018
data_2018 = get_movie_data('2018')
data = data.append(data_2018,ignore_index=True)

In [ ]:
#collecting movies of 2019
data_2019 = get_movie_data('2019')
data = data.append(data_2019,ignore_index=True)

In [ ]:
#collecting movies of 2020
data_2020 = get_movie_data('2020')
data = data.append(data_2020,ignore_index=True)

In [ ]:
#collecting movies of 2021
data_2021 = get_movie_data('2021')
data = data.append(data_2021,ignore_index=True)

In [ ]:
#collecting movies of 2022
data_2022 = get_movie_data('2022')
data = data.append(data_2022,ignore_index=True)

In [ ]:
data

,id,movie_title,keywords,tagline,overview,director,actor_1,actor_2,actor_3,genres,popularity,year,vote_average,vote_count
0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000,NaN,NaN
1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000,NaN,NaN
2,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000,NaN,NaN
3,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000,NaN,NaN
4,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2649,796207.0,Dobaaraa,NaN,NaN,NaN,Anurag Kashyap,Taapsee PannuPavail Gulati,NaN,NaN,Science Fiction Thriller Horror,0.932,2022,NaN,NaN
2650,NaN,Liger[a],NaN,NaN,NaN,Puri Jagannadh,Vijay DeverakondaAnanya PandeyMike Tyson,NaN,NaN,NaN,NaN,2022,NaN,NaN
2651,0.0,Thai Massage,NaN,NaN,NaN,Mangesh Hadawale,DivyennduGajraj RaoRajpal YadavSunny HindujaVi...,NaN,NaN,NaN,NaN,2022,NaN,NaN
2652,496331.0,Brahmāstra,superhero,The God of all weapons.,NaN,Ayan Mukerji,Amitabh BachchanRanbir KapoorAlia BhattMouni R...,NaN,NaN,Adventure Action Fantasy,1.789,2022,NaN,NaN


Data Preprocessing 

In [ ]:
data.isna().sum()

id               186
movie_title      274
keywords        1906
tagline         2071
overview         763
director         452
actor_1          467
actor_2          972
actor_3         1243
genres           910
popularity       742
year               0
vote_average    1031
vote_count      1031
dtype: int64

In [ ]:
data1 = data.drop(['tagline'], axis = 1, inplace=True)

In [ ]:
data = data1 

In [ ]:
data2 = data.drop(['keywords'], axis = 1, inplace=True)

In [ ]:
data = data[data['id']!=0.0]

In [ ]:
data = data[data['id']!=np.NaN]
data

,id,movie_title,overview,director,actor_1,actor_2,actor_3,genres,popularity,year,vote_average,vote_count
23,11518.0,Mohabbatein,"At a prestigious all-male university, three fr...",NaN,NaN,NaN,NaN,Drama Romance,9.734,2000,7.0,171.0
24,16987.0,Kaho Naa... Pyaar Hai,Life is paradise for an upright young man and ...,NaN,NaN,NaN,NaN,Romance Drama,6.129,2000,6.9,97.0
25,46406.0,Mission Kashmir,A police officer adopts the son and sole survi...,NaN,NaN,NaN,NaN,Action Drama Thriller,3.105,2000,6.3,41.0
26,161223.0,Dulhan Hum Le Jayenge,Sapna has lost her parents at an early age and...,NaN,NaN,NaN,NaN,Comedy Romance,2.460,2000,5.7,21.0
27,166231.0,Refugee,The films tells the story of an un-named India...,NaN,NaN,NaN,NaN,Drama Music Romance,2.607,2000,6.1,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2648,601635.0,Laal Singh Chaddha,Events in India's history unfold through the p...,Advait Chandan,Aamir KhanKareena KapoorNaga Chaitanya,NaN,NaN,Drama Comedy Romance,2.586,2022,NaN,NaN
2649,796207.0,Dobaaraa,NaN,Anurag Kashyap,Taapsee PannuPavail Gulati,NaN,NaN,Science Fiction Thriller Horror,0.932,2022,NaN,NaN
2650,NaN,Liger[a],NaN,Puri Jagannadh,Vijay DeverakondaAnanya PandeyMike Tyson,NaN,NaN,NaN,NaN,2022,NaN,NaN
2652,496331.0,Brahmāstra,NaN,Ayan Mukerji,Amitabh BachchanRanbir KapoorAlia BhattMouni R...,NaN,NaN,Adventure Action Fantasy,1.789,2022,NaN,NaN


In [1]:
data = data[data['id'].isna() == False]

NameError: name 'data' is not defined

In [2]:
data.isna().sum()

NameError: name 'data' is not defined

In [ ]:
data = data[data['actor_1'].isna() == False]

In [ ]:
data.isna().sum()

id                0
movie_title       0
overview         20
director         13
actor_1           0
actor_2         434
actor_3         638
genres          161
popularity        0
year              0
vote_average    268
vote_count      268
dtype: int64

In [ ]:
data = data[data['director'].isna()==False]

In [ ]:
data = data[data['overview'].isna()==False]

In [ ]:
data.isna().sum()

id                0
movie_title       0
overview          0
director          0
actor_1           0
actor_2         421
actor_3         617
genres          151
popularity        0
year              0
vote_average    245
vote_count      245
dtype: int64

In [ ]:
data = data[data['genres'].isna()==False]

In [ ]:
data.isna().sum()

id                0
movie_title       0
overview          0
director          0
actor_1           0
actor_2         398
actor_3         571
genres            0
popularity        0
year              0
vote_average    177
vote_count      176
dtype: int64

In [ ]:
data = data[data['actor_2'].isna()==False]

In [ ]:
data.isna().sum()

id                0
movie_title       0
overview          0
director          0
actor_1           0
actor_2           0
actor_3         173
genres            0
popularity        0
year              0
vote_average    120
vote_count      120
dtype: int64

In [ ]:
df = data

In [ ]:
df['actor_3'] = df['actor_3'].replace(np.nan, 'unknown')
df['movie_title'] = df['movie_title'].str.lower()
df['describe'] = df['actor_1'] + ' ' + df['actor_2'] + ' '+ df['actor_3'] + ' '+ df['director'] + ' ' + df['genres'] + ' '  + df['popularity'].map(str)

In [ ]:
df

,id,movie_title,overview,director,actor_1,actor_2,actor_3,genres,popularity,year,vote_average,vote_count,describe
35,418655.0,anjaane,Laxmi finds an abandoned baby at a temple and ...,Ravi Rai,Raveena Tandon,Vivek Mushran,unknown,Drama,0.600,2000,10.0,1.0,Raveena Tandon Vivek Mushran unknown Ravi Rai ...
37,413278.0,apradhi kaun,"A wealthy man, suspected of murder, is killed ...",Mohan Bhakri,Ishrat Ali,Shagufta Ali,unknown,Thriller Mystery Drama,0.600,2000,8.0,1.0,Ishrat Ali Shagufta Ali unknown Mohan Bhakri T...
39,302309.0,baaghi,"Growing up in Mumbai, Surya is in awe of a pow...",Rajesh Kumar Singh,Sanjay Dutt,Manisha Koirala,unknown,Romance Crime,0.840,2000,4.5,2.0,Sanjay Dutt Manisha Koirala unknown Rajesh Kum...
43,11854.0,kuch kuch hota hai,Anjali is left heartbroken when her best frien...,Karan Johar,Shahrukh Khan,Kajol,Rani Mukerji,Romance Drama,13.007,2000,7.7,309.0,Shahrukh Khan Kajol Rani Mukerji Karan Johar R...
45,271484.0,beti no.1,This warmhearted comedy takes place in a house...,Rama Rao Tatineni,Govinda,Rambha,unknown,Comedy Drama Family Romance,0.711,2000,3.5,4.0,Govinda Rambha unknown Rama Rao Tatineni Comed...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2168,454776.0,fukrey returns,"A year after the first movie, the four friends...",Mrighdeep Singh Lamba,Pulkit Samrat,Priya Anand,Ali Fazal,Comedy Drama,2.262,2017,6.6,24.0,Pulkit Samrat Priya Anand Ali Fazal Mrighdeep ...
2170,496626.0,sallu ki shaadi,Sallu ki Shaadi is an upcoming Indian romantic...,Mohammad Israr Ansari,Zeenat Aman,Kiran Kumar,Asrani,Comedy,0.600,2017,NaN,NaN,Zeenat Aman Kiran Kumar Asrani Mohammad Israr ...
2171,194522.0,monsoon shootout,A rookie cop joins the crime branch of the Mum...,Amit Kumar,Nawazuddin Siddiqui,Vijay Varma & Tannishtha Chatterjee,unknown,Thriller,1.465,2017,6.6,14.0,Nawazuddin Siddiqui Vijay Varma & Tannishtha C...
2192,494211.0,3 storeys,3 Storeys is an 2018 Hindi thriller drama film...,Arjun Mukerjee,"Richa Chaddha,Pulkit Samrat,Sharman Joshi,Renu...","Masumeh Makhija,Laksh Singh,Aisha Ahmed,Ankit ...",unknown,Drama,0.600,2018,6.7,3.0,"Richa Chaddha,Pulkit Samrat,Sharman Joshi,Renu..."


In [ ]:
df.to_csv('bollywood_data.csv',index=False)